# 大模型

## 模型支持

### OpenAI

In [1]:
from textlong.llm import openai
from textlong.io import log

log(
    openai,
    [{"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}],
    model="gpt-3.5-turbo"
)

小兔子闭上眼，做着甜美的梦，
甜甜的草莓味，让它迷失在其中。
梦里有欢乐的游戏，和朋友们一起玩，
睡醒了它还在微笑，因为梦想成真的感觉。



{'output': '小兔子闭上眼，做着甜美的梦，\n甜甜的草莓味，让它迷失在其中。\n梦里有欢乐的游戏，和朋友们一起玩，\n睡醒了它还在微笑，因为梦想成真的感觉。',
 'tools_call': {}}

In [2]:
from textlong.llm.tools import tool, convert_to_openai_tool
from textlong.llm import openai
from textlong.io import log
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

log(openai, "今天广州天气如何啊", tools=[convert_to_openai_tool(get_current_weather)], model="gpt-3.5-turbo")

[TOOLS_CALL] {"index": 0, "function": {"id": "call_2YNg3iXT0zqdFc11RUAaqIrD", "type": "function", "name": "get_current_weather", "arguments": ""}}
[TOOLS_CALL] {"index": 0, "function": {"id": "", "type": "function", "name": null, "arguments": "{\n"}}
[TOOLS_CALL] {"index": 0, "function": {"id": "", "type": "function", "name": null, "arguments": " "}}
[TOOLS_CALL] {"index": 0, "function": {"id": "", "type": "function", "name": null, "arguments": " \""}}
[TOOLS_CALL] {"index": 0, "function": {"id": "", "type": "function", "name": null, "arguments": "location"}}
[TOOLS_CALL] {"index": 0, "function": {"id": "", "type": "function", "name": null, "arguments": "\":"}}
[TOOLS_CALL] {"index": 0, "function": {"id": "", "type": "function", "name": null, "arguments": " \""}}
[TOOLS_CALL] {"index": 0, "function": {"id": "", "type": "function", "name": null, "arguments": "广"}}
[TOOLS_CALL] {"index": 0, "function": {"id": "", "type": "function", "name": null, "arguments": "州"}}
[TOOLS_CALL] {"index":

{'output': '',
 'tools_call': {0: {'index': 0,
   'function': {'id': 'call_2YNg3iXT0zqdFc11RUAaqIrD',
    'type': 'function',
    'name': None,
    'arguments': '{\n  "location": "广州"\n}'}}}}

### 智谱AI

In [3]:
from textlong.llm import zhipu
from textlong.io import log

log(
    zhipu,
    [{"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}],
    model="glm-4-flash"
)

小兔子乖乖梦儿甜，
梦中草原绿油油，
月亮当灯星星点，
梦回田野跑欢欢。



{'output': '小兔子乖乖梦儿甜，\n梦中草原绿油油，\n月亮当灯星星点，\n梦回田野跑欢欢。', 'tools_call': {}}

In [4]:
from textlong.llm.tools import tool, convert_to_openai_tool
from textlong.llm import zhipu
from textlong.io import log
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

log(zhipu, "今天广州天气如何啊", tools=[convert_to_openai_tool(get_current_weather)], model="glm-4-air")

好的，我可以帮你查询广州的天气。请问你需要我调用哪个API呢？



{'output': '好的，我可以帮你查询广州的天气。请问你需要我调用哪个API呢？', 'tools_call': {}}

### 通义千问

In [5]:
from textlong.llm import qwen
from textlong.io import log

log(
    qwen,
    [{"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}],
    model="qwen2-1.5b-instruct"
)

小兔子，梦中跳，  
胡萝卜，满天空飘。  
月亮船，带它逛，  
醒来笑，梦多妙。



{'output': '小兔子，梦中跳，  \n胡萝卜，满天空飘。  \n月亮船，带它逛，  \n醒来笑，梦多妙。', 'tools_call': {}}

In [6]:
from textlong.llm.tools import tool, convert_to_openai_tool
from textlong.llm import qwen
from textlong.io import log
import json

@tool
def get_current_weather(location: str):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

log(qwen, "今天广州天气如何啊", tools=[convert_to_openai_tool(get_current_weather)])

[TOOLS_CALL] {"index": 0, "id": "call_0b9e6473108047529b24f0", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
[TOOLS_CALL] {"index": 0, "id": "", "type": "function", "function": {}}
{"0": {"index": 0, "id": "call_0b9e6473108047529b24f0", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}


{'output': '',
 'tools_call': {0: {'index': 0,
   'id': 'call_0b9e6473108047529b24f0',
   'type': 'function',
   'function': {'name': 'get_current_weather',
    'arguments': '{"location": "广州"}'}}}}

## Fake

### fake_llm: 模拟大模型调用过程

这可以观察提示语等内部结构。

In [7]:
from textlong.llm import fake_llm
from textlong.io import log

log(
    fake_llm,
    [
        {"role": "system", "content": "你是一个写作能手。"},
        {"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}
    ],
    model="qwen2-1.5b-instruct"
)

[INFO] system: 你是一个写作能手。
[INFO] user: 你能帮我写一首关于兔子做梦的四句儿歌?
这是一个模拟调用!



{'output': '这是一个模拟调用!', 'tools_call': {}}